<a href="https://colab.research.google.com/github/FeelOneE/price_prediction/blob/main/coin_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/스터디/코인 트레이드/data


/content/drive/MyDrive/스터디/코인 트레이드/data


In [4]:
%ls -al

total 3758
-rw------- 1 root root       0 Dec 19 07:57 train.csv
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [5]:
import pandas as pd

data = pd.read_csv("XRP.csv")
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,2021-12-18T00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,2021-12-18T00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,2021-12-17T23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,2021-12-17T23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,2021-12-17T23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [6]:
# 시간 T를 기준으로 나누어 날짜는 제외하고 시간 분만 남게 처리
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split('T', expand=True).apply(lambda x : x[1], axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [7]:
# 시간을 분으로 치환
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split(':', expand=True).apply(lambda x : (int(x[0])*60)+(int(x[1])) , axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,5,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,0,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,1435,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,1430,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,1425,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [8]:
# 안쓰는 컬럼 삭제 market, utc_time, unit, timestamp
data.drop(['market','candle_date_time_utc','unit','timestamp'], axis=1, inplace=True)
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,5,986.0,987.0,984.0,987.0,1.785798e+09,1.812305e+06
1,0,989.0,989.0,981.0,986.0,2.128017e+09,2.161762e+06
2,1435,984.0,989.0,983.0,989.0,2.275873e+09,2.311628e+06
3,1430,987.0,990.0,984.0,985.0,3.238981e+09,3.281353e+06
4,1425,983.0,987.0,981.0,987.0,5.052325e+09,5.141900e+06


In [9]:
# price, volume 소수점 5자리까지만 
data['candle_acc_trade_price'] = data.candle_acc_trade_price.apply(lambda x : '%.5f' % x)
data['candle_acc_trade_volume'] = data.candle_acc_trade_volume.apply(lambda x : '%.5f' % x)
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
0,5,986.0,987.0,984.0,987.0,1785797775.07049,1812305.45733
1,0,989.0,989.0,981.0,986.0,2128016988.39383,2161761.89739
2,1435,984.0,989.0,983.0,989.0,2275873191.28294,2311628.47649
3,1430,987.0,990.0,984.0,985.0,3238981389.51325,3281353.39430
4,1425,983.0,987.0,981.0,987.0,5052324608.41463,5141900.24199


In [10]:
# 시간순서대로 데이터 다시 정렬
data = data[::-1]
data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
30199,1240,1475.0,1480.0,1470.0,1480.0,2449703432.48617,1661509.79377
30198,1245,1480.0,1480.0,1470.0,1480.0,1401303636.71963,949623.96941
30197,1250,1475.0,1480.0,1475.0,1480.0,964682189.53982,652588.80397
30196,1255,1480.0,1480.0,1475.0,1475.0,685879546.75352,464264.68260
30195,1260,1475.0,1480.0,1475.0,1480.0,519939889.71916,351858.39885


In [12]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
std_data = data.copy()
std_data[:] = scaler.fit_transform(data[:])
std_data.head(5)

,candle_date_time_kst,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume
30199,0.864111,0.784483,0.783237,0.803922,0.791667,0.022750,0.021775
30198,0.867596,0.791667,0.783237,0.803922,0.791667,0.013011,0.012443
30197,0.871080,0.784483,0.783237,0.810458,0.791667,0.008956,0.008549
30196,0.874564,0.791667,0.783237,0.810458,0.784483,0.006366,0.006081
30195,0.878049,0.784483,0.783237,0.810458,0.791667,0.004825,0.004607


In [13]:
# 학습된 정규화로 로우 데이터 정규화 수행
#data[:2]
scaler.transform(data[:2])

array([[0.8641115 , 0.78448276, 0.78323699, 0.80392157, 0.79166667,
        0.02274972, 0.02177474],
       [0.86759582, 0.79166667, 0.78323699, 0.80392157, 0.79166667,
        0.01301138, 0.01244297]])

In [23]:
!pip install sklearn

In [25]:
import joblib 
# 객체를 pickled binary file 형태로 저장한다 
file_name = 'scaler.pkl' 
joblib.dump(scaler, file_name) 

['scaler.pkl']

In [ ]:
import joblib 
# pickled binary file 형태로 저장된 객체를 로딩한다 
file_name = 'scaler.pkl' 
obj = joblib.load(file_name) 

In [26]:
%ls -al

total 3759
-rw------- 1 root root    1349 Dec 19 09:40 scaler.pkl
-rw------- 1 root root       0 Dec 19 07:57 train.csv
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [14]:
# 정답 라벨 붙이기
# 데이터를 60개로 묶고, 마지막 종가(trade_price)를 기준으로 
# 다음 60개의 데이터 중 최대 값이 종가의 1.035배 이상인 적이 있는지 찾기
import numpy as np

data_set = []
x_data = []
y_data = []
def find_label(index):
  tmp = []
  sample = data[index:index+60]
  next_sample = data[index+60:index+120]

  trade_price = sample['trade_price'].iloc[-1]
  x_data.append(scaler.transform(sample))
  #print(trade_price)

  price_list = []
  price_list.extend(next_sample['opening_price'])
  price_list.extend(next_sample['trade_price'])

  max_price = max(price_list)
  #print(max_price)

  expect_price = trade_price*1.03
  #print(expect_price)

  if max_price >= expect_price :
    y_data.append(1)

    return 1
    # print(trade_price)
    # print(expect_price)
    # print(max_price)
    # print("find label! : ", index)
    # print('-'*30)
  
  y_data.append(0)
  return 0

In [15]:
# 전체 데이터와 정답 레이블의 비율
total_cnt = 0
cnt = 0
skip_size = 1
for idx in range(0, len(data)-120, skip_size):
  cnt += find_label(idx)
  total_cnt += 1

rate = float(cnt / total_cnt)
print(rate)


0.10302526595744681


In [16]:
import numpy as np
x_data_np = np.array(x_data)
y_data_np = np.array(y_data)
x_data_np.shape, y_data_np.shape

((30080, 60, 7), (30080,))

In [17]:
import numpy as np
from keras.layers import Conv1D, Activation, MaxPool2D, Flatten, Dense
from keras.models import Sequential
from keras.datasets import mnist

In [18]:
# Images fed into this model are 512 x 512 pixels with 3 channels

data_shape = (60,7,1)
#data_shape = (28,28,1)

# Set up the model

model = Sequential()

# Add convolutional layer with 3, 3 by 3 filters and a stride size of 1
# Set padding so that input size equals output size

model.add(Conv1D(4,3,input_shape=data_shape))
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60, 5, 4)          16        
                                                                 
 flatten (Flatten)           (None, 1200)              0         
                                                                 
 dense (Dense)               (None, 10)                12010     
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 12,158
Trainable params: 12,158
Non-trainable params: 0
_________________________________________________________________


In [19]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data_np, y_data_np, test_size=0.4, shuffle=True, random_state=1004 )

# 모델에 입력가능하게 차원을 축소(?)함
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
x_train.shape, x_test.shape

x_train.shape,x_test.shape,y_train.shape,y_test.shape


((18048, 60, 7, 1), (12032, 60, 7, 1), (18048,), (12032,))

In [20]:
# 모델 학습

model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test) )

Epoch 1/10
564/564 [==============================] - 5s 6ms/step - loss: 0.3343 - accuracy: 0.8963 - val_loss: 0.3184 - val_accuracy: 0.8979
Epoch 2/10
564/564 [==============================] - 3s 5ms/step - loss: 0.3253 - accuracy: 0.8963 - val_loss: 0.3145 - val_accuracy: 0.8979
Epoch 3/10
564/564 [==============================] - 3s 5ms/step - loss: 0.3204 - accuracy: 0.8955 - val_loss: 0.3090 - val_accuracy: 0.8994
Epoch 4/10
564/564 [==============================] - 3s 6ms/step - loss: 0.3177 - accuracy: 0.8973 - val_loss: 0.3138 - val_accuracy: 0.9004
Epoch 5/10
564/564 [==============================] - 3s 5ms/step - loss: 0.3160 - accuracy: 0.8974 - val_loss: 0.3094 - val_accuracy: 0.8988
Epoch 6/10
564/564 [==============================] - 3s 5ms/step - loss: 0.3133 - accuracy: 0.8982 - val_loss: 0.3052 - val_accuracy: 0.8996
Epoch 7/10
564/564 [==============================] - 3s 5ms/step - loss: 0.3133 - accuracy: 0.8989 - val_loss: 0.3052 - val_accuracy: 0.8991
Epoch 

In [21]:
model.predict(x_test[:3])

array([[0.957903  , 0.04209694],
       [0.9494023 , 0.05059777],
       [0.97378767, 0.02621236]], dtype=float32)

In [28]:
model.save('cnn_prediction')

INFO:tensorflow:Assets written to: cnn_prediction/assets


In [29]:
%ls -al

total 3763
drwx------ 4 root root    4096 Dec 19 09:41 cnn_prediction/
-rw------- 1 root root    1349 Dec 19 09:40 scaler.pkl
-rw------- 1 root root       0 Dec 19 07:57 train.csv
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [30]:
%pwd

'/content/drive/My Drive/스터디/코인 트레이드/data'